# Initialize

In [1]:
# Options

## Debugging level
debug_level = 0


In [2]:
# Initialize framework
constants, tools = init_preprocessing(debug_level)
%matplotlib inline


In [3]:
# Read anonymized data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)


In [4]:
# Build lookup tables from the data
lut_address_state = forecast["Mailing State/Province"].drop_duplicates().to_dict()
lut_record_type = forecast["Organization Record Type"].drop_duplicates().to_dict()
lut_customer_type = forecast["ATLAS Customer Type"].drop_duplicates().to_dict()
lut_vendor_id = revenue2020A["Vendor ID"].drop_duplicates().to_dict()
lut_address = atlas["Address"].drop_duplicates().to_dict()
lut_address_lat = atlas["Lat"].drop_duplicates().to_dict()
lut_address_lon = atlas["Long"].drop_duplicates().to_dict()

# Write lookup tables to file
tools.tool_lookup_tables.write_lookup_table(lut_address_state, constants.LUT_ADDRESS_STATE_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_record_type, constants.LUT_RECORD_TYPE_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_customer_type, constants.LUT_CUSTOMER_TYPE_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_vendor_id, constants.LUT_VENDOR_ID_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_address, constants.LUT_ADDRESS_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_address_lat, constants.LUT_ADDRESS_LAT_FILE_PATH)
tools.tool_lookup_tables.write_lookup_table(lut_address_lon, constants.LUT_ADDRESS_LON_FILE_PATH)


In [5]:
# Read lookup tables
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_STATE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_RECORD_TYPE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_CUSTOMER_TYPE_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_VENDOR_ID_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_LAT_FILE_PATH)
# tool_lookup_tables.read_lookup_table( constants.LUT_ADDRESS_LON_FILE_PATH)


In [6]:
# Build combined data table

## Get list of customers to use as the index
# customer_list = tools.tool_mapping.read_list(constants.LIST_CUSTOMER_NAME_LIST_FILE_PATH)
# invoice_data = pd.DataFrame(index=customer_list)
# # invoice_data.append(atlas.loc[invoice_data.index])
# atlas.loc[ invoice_data.reset_index()["index"][0] ]
# # invoice_data.reset_index()["Customers"].apply(lambda x: atlas.loc[x])

## lut_address_state: Names do not match; redundant data.
# atlas["State"] = atlas.reset_index()["Customers"].apply(lambda x: lut_address_state.get(x, ""))
# atlas.State.dropna()

## lut_record_type: Not enough data.

## lut_customer_type: Not enough data.

## lut_vendor_id: Not enough data.

## lut_address, lut_address_lat, lut_address_lon: Already in the data.

## Export to PREPROCESSED_COMBINED_DATA_FILE_PATH


# invoice_data.head(1)

In [7]:
# Scratchwork

# df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
#                    columns=['a', 'b', 'c'])

# df2.loc[0]
# df2

# atlas.loc[] 

# y = invoice_data.reset_index()["index"][0]
# z='Louisiana Board Of Regents'
# atlas.loc[z]
# atlas.append()


# customer_list


In [8]:
# Extract existing data into additional columns
invoice_data_by_customer = atlas.copy()

# Remove spaces from columns
invoice_data_by_customer.columns = [ "NumberOfUsers", "InvoiceDate", "InvoiceNumber", "InvoiceAmount", "Subscription", "AccountCode", "Address", "Lat", "Long", "ServiceStart", "ServiceEnd", "SubscriptionDuration" ]

# Address
# invoice_data_by_customer["Address_State"] = invoice_data.Address.State

# Service delay
invoice_data_by_customer["ServiceDelay"] = (invoice_data_by_customer["InvoiceDate"] - invoice_data_by_customer["ServiceStart"]).apply(lambda date: date.days)

# Extract y/m/q
invoice_data_by_customer = tools.tool_data_processing.extract_and_append_year_month_quarter(invoice_data_by_customer, "InvoiceDate")
invoice_data_by_customer = tools.tool_data_processing.extract_and_append_year_month_quarter(invoice_data_by_customer, "ServiceStart")
invoice_data_by_customer = tools.tool_data_processing.extract_and_append_year_month_quarter(invoice_data_by_customer, "ServiceEnd")

# Build historical views
invoice_data_by_service_start   = invoice_data_by_customer.copy().reset_index().set_index("ServiceStart").sort_index(ascending=True)
invoice_data_by_invoice_date    = invoice_data_by_customer.copy().reset_index().set_index("InvoiceDate").sort_index(ascending=True)


In [9]:
# Fix forecast data
forecast.columns = [ "Address_State", "OrganizationRecordType",
    "ATLASCustomerType", "SubscriptionFee", "SubscriptionType",
    "ContractStartDate", "ContractEndDate", "SubscriptionDuration" ]
forecast = tools.tool_data_processing.extract_and_append_year_month_quarter(forecast, "ContractStartDate")

# Fix revenue2020A data
revenue2020A.columns = [ "InvoiceDate", "InvoiceNumber", "VendorID", "InvoiceAmount",
    "AccountCode", "MailDate", "AmountPaid", " PymtReceived",
    "ServiceStart", "ServiceEnd", "ServiceDuration"]
revenue2020A = tools.tool_data_processing.extract_and_append_year_month_quarter(revenue2020A, "ServiceStart")


In [10]:
# Export
invoice_data_by_customer.to_pickle(constants.PREPROCESSED_INVOICE_CUSTOMER_FILE_PATH)
invoice_data_by_service_start.to_pickle(constants.PREPROCESSED_INVOICE_SERVICEDATE_FILE_PATH)
invoice_data_by_invoice_date.to_pickle(constants.PREPROCESSED_INVOICE_INVOICEDATE_FILE_PATH)
forecast.to_pickle(constants.PREPROCESSED_FORECAST_CLEANED_FILE_PATH)
revenue2020A.to_pickle(constants.PREPROCESSED_REVENUE2020A_CLEANED_FILE_PATH)
